In [1]:
! pip install tranformer
! pip install evaluate

ERROR: Could not find a version that satisfies the requirement tranformer (from versions: none)
ERROR: No matching distribution found for tranformer
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.4 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer


In [3]:
raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
raw_datasets

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

mrpc/train-00000-of-00001.parquet:   0%|          | 0.00/649k [00:00<?, ?B/s]

mrpc/validation-00000-of-00001.parquet:   0%|          | 0.00/75.7k [00:00<?, ?B/s]

mrpc/test-00000-of-00001.parquet:   0%|          | 0.00/308k [00:00<?, ?B/s]

...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [4]:
def tokenize_function(example):
    return tokenizer(example["sentence1"], example["sentence2"], truncation=True)


tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/3668 [00:00<?, ? examples/s]

Map:   0%|          | 0/408 [00:00<?, ? examples/s]

Map:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [5]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

In [6]:
# print(model.forward.__doc__)

In [7]:
tokenized_datasets["train"]["input_ids"]

Column([[101, 2572, 3217, 5831, 5496, 2010, 2567, 1010, 3183, 2002, 2170, 1000, 1996, 7409, 1000, 1010, 1997, 9969, 4487, 23809, 3436, 2010, 3350, 1012, 102, 7727, 2000, 2032, 2004, 2069, 1000, 1996, 7409, 1000, 1010, 2572, 3217, 5831, 5496, 2010, 2567, 1997, 9969, 4487, 23809, 3436, 2010, 3350, 1012, 102], [101, 9805, 3540, 11514, 2050, 3079, 11282, 2243, 1005, 1055, 2077, 4855, 1996, 4677, 2000, 3647, 4576, 1999, 2687, 2005, 1002, 1016, 1012, 1019, 4551, 1012, 102, 9805, 3540, 11514, 2050, 4149, 11282, 2243, 1005, 1055, 1999, 2786, 2005, 1002, 6353, 2509, 2454, 1998, 2853, 2009, 2000, 3647, 4576, 2005, 1002, 1015, 1012, 1022, 4551, 1999, 2687, 1012, 102], [101, 2027, 2018, 2405, 2019, 15147, 2006, 1996, 4274, 2006, 2238, 2184, 1010, 5378, 1996, 6636, 2005, 5096, 1010, 2002, 2794, 1012, 102, 2006, 2238, 2184, 1010, 1996, 2911, 1005, 1055, 5608, 2018, 2405, 2019, 15147, 2006, 1996, 4274, 1010, 5378, 1996, 14792, 2005, 5096, 1012, 102], [101, 2105, 6021, 19481, 13938, 2102, 1010, 21628,

In [8]:
# this things are automatically handel by the traner of hugging face but we are using the core pytorch training loop we have to handle this manually
tokenized_datasets = tokenized_datasets.remove_columns(["sentence1", "sentence2", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

In [9]:
type(tokenized_datasets["train"]["input_ids"][0])

torch.Tensor

In [10]:
tokenized_datasets["train"].column_names

['labels', 'input_ids', 'token_type_ids', 'attention_mask']

In [11]:
# now we have to make batches
from torch.utils.data import DataLoader

train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=16, collate_fn=data_collator
)

test_dataloader = DataLoader(
    tokenized_datasets["test"], batch_size=16, collate_fn=data_collator
)

valid_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=16, collate_fn=data_collator
)

In [12]:
for index, batch in enumerate(valid_dataloader):
    if index < 4:
        print(batch["input_ids"].shape)
        # the dynamic padding is working

torch.Size([16, 78])
torch.Size([16, 74])
torch.Size([16, 73])
torch.Size([16, 77])


In [13]:
for batch in train_dataloader:
    break
{k: v.shape for k, v in batch.items()}

{'labels': torch.Size([16]),
 'input_ids': torch.Size([16, 72]),
 'token_type_ids': torch.Size([16, 72]),
 'attention_mask': torch.Size([16, 72])}

In [14]:
batch.items()

ItemsView({'labels': tensor([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1]), 'input_ids': tensor([[  101,  1044,  1012,  ...,  3431,  1012,   102],
        [  101,  5758,  2036,  ...,     0,     0,     0],
        [  101,  8997,  5045,  ...,     0,     0,     0],
        ...,
        [  101,  3904,  1997,  ...,     0,     0,     0],
        [  101,  2845, 14331,  ...,   102,     0,     0],
        [  101,  2149,  4614,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 1, 1, 1],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 1, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])})

In [15]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
output = model(**batch)
output

SequenceClassifierOutput(loss=tensor(1.0930, grad_fn=<NllLossBackward0>), logits=tensor([[ 0.7588, -0.2560],
        [ 0.7061, -0.2262],
        [ 0.7079, -0.2133],
        [ 0.7161, -0.2276],
        [ 0.7219, -0.2280],
        [ 0.7097, -0.2365],
        [ 0.7106, -0.2318],
        [ 0.7013, -0.2136],
        [ 0.7190, -0.2345],
        [ 0.7356, -0.2332],
        [ 0.6947, -0.2193],
        [ 0.7117, -0.2308],
        [ 0.6995, -0.2274],
        [ 0.7204, -0.2379],
        [ 0.7292, -0.2066],
        [ 0.7186, -0.2144]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [17]:
print(output.loss,output.logits)

tensor(1.0930, grad_fn=<NllLossBackward0>) tensor([[ 0.7588, -0.2560],
        [ 0.7061, -0.2262],
        [ 0.7079, -0.2133],
        [ 0.7161, -0.2276],
        [ 0.7219, -0.2280],
        [ 0.7097, -0.2365],
        [ 0.7106, -0.2318],
        [ 0.7013, -0.2136],
        [ 0.7190, -0.2345],
        [ 0.7356, -0.2332],
        [ 0.6947, -0.2193],
        [ 0.7117, -0.2308],
        [ 0.6995, -0.2274],
        [ 0.7204, -0.2379],
        [ 0.7292, -0.2066],
        [ 0.7186, -0.2144]], grad_fn=<AddmmBackward0>)


In [18]:
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=5e-5)

In [19]:
# explain this  what this do?
from transformers import get_scheduler
num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)  # optimizer updates or optimizer steps

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)
print(num_training_steps)




690


In [20]:
# now check for the device
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
device

device(type='cuda')

In [21]:
# from tqdm.auto import tqdm
# progress_bar = tqdm(range(num_training_steps))
# # now we have to train our model

# model.train()
# for epoch in range(num_epochs):
#     for batch in train_dataloader:
#         batch = {k: v.to(device) for k, v in batch.items()}
#         outputs = model(**batch)
#         loss = outputs.loss
#         loss.backward() # this is for back propigation

#         optimizer.step()
#         lr_scheduler.step()
#         optimizer.zero_grad()
#         progress_bar.update(1)


In [24]:
import evaluate
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_epochs):
    # ===== Training =====
    model.train()
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # ===== Evaluation =====
    model.eval()
    metric = evaluate.load("glue", "mrpc")  # re-init each epoch

    for batch in valid_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        metric.add_batch(predictions=predictions, references=batch["labels"])

    # compute after full epoch
    eval_results = metric.compute()
    print(f"Epoch {epoch+1}: {eval_results}")


  0%|          | 0/690 [00:00<?, ?it/s]

Epoch 1: {'accuracy': 0.8504901960784313, 'f1': 0.8953687821612349}
Epoch 2: {'accuracy': 0.8504901960784313, 'f1': 0.8953687821612349}
Epoch 3: {'accuracy': 0.8504901960784313, 'f1': 0.8953687821612349}


In [25]:
model.eval()
for batch in test_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.8133333333333334, 'f1': 0.8652719665271966}